# Visualising Gun Violence With Folium

In [ ]:
In this piece we'll be using the Folium package visualise gun violence in  to create interactive maps '

In [1]:
import pandas as pd
import folium

To begin, let's load the data. I've preprocessed the original kaggle dataset so it only contains data from 2017. I'll be also using population data from the [U.S. Census Beaureau](https://www.census.gov/data/tables/2017/demo/popest/state-total.html) to calculate incident rates. 

In [3]:
df = pd.read_csv('2017_gun_data.csv')
population = pd.read_csv('2017 population data.csv')
df.head()

,incident_id,state,city_or_county,n_killed,n_injured,incident_url,gun_stolen,gun_type,latitude,longitude,n_guns_involved,year,month_year
0,740176,Tennessee,Memphis,1,0,http://www.gunviolencearchive.org/incident/740176,0::Unknown,0::Unknown,35.0670,-89.9156,1.0,2017,2017-01
1,741857,Nebraska,Omaha,0,0,http://www.gunviolencearchive.org/incident/741857,0::Unknown,0::Unknown,41.2960,-95.9470,1.0,2017,2017-01
2,741852,Nebraska,Omaha,0,1,http://www.gunviolencearchive.org/incident/741852,0::Unknown,0::Unknown,41.1918,-95.9528,1.0,2017,2017-01
3,740911,Pennsylvania,Erie,0,1,http://www.gunviolencearchive.org/incident/740911,0::Unknown,0::Unknown,42.1306,-80.0272,1.0,2017,2017-01
4,744584,New Mexico,Portales,0,0,http://www.gunviolencearchive.org/incident/744584,0::Unknown,0::Unknown,34.1898,-103.3240,1.0,2017,2017-01


In [4]:
population.head()

,state,2017 Population
0,Alabama,4874747
1,Alaska,739795
2,Arizona,7016270
3,Arkansas,3004279
4,California,39536653


To visualise the choropleth, We'll need to merge the population data and the gun violence data so that we can calculate the number of incidents per 100,000 people:

In [5]:
grouped = df.groupby(['state'])[['n_killed', 'n_injured']].sum()
choro_df= pd.merge(population, grouped, right_index=True, left_on='state')
choro_df.head()

,state,2017 Population,n_killed,n_injured
0,Alabama,4874747,543,856
1,Alaska,739795,68,66
2,Arizona,7016270,280,294
3,Arkansas,3004279,214,430
4,California,39536653,1383,1911


Let's now create new columns containing the incident rates:

In [6]:
choro_df['killed per 100000'] = (choro_df['n_killed'] / choro_df['2017 Population']) * 100000
choro_df['incidents per 100000'] = (choro_df['n_killed'] + choro_df['n_injured'])/ choro_df['2017 Population'] * 100000
choro_df.head()

,state,2017 Population,n_killed,n_injured,killed per 100000,incidents per 100000
0,Alabama,4874747,543,856,11.139040,28.698925
1,Alaska,739795,68,66,9.191736,18.113126
2,Arizona,7016270,280,294,3.990724,8.180985
3,Arkansas,3004279,214,430,7.123173,21.436092
4,California,39536653,1383,1911,3.498020,8.331509


Now it's time to visualise the choropleths. To do so, we first create a map object in folium:

In [12]:
n_killed_map = folium.Map(location=[38.0215482, -95.471191], tiles='OpenStreetMap', zoom_start=4.5)

Then we need to add the choropleth to the map. To do that we'll need to pass a geojson file to the choropleth function, which contains the co-ordinates for the state boundaries. We'll also need to pass the gun violence data to the function as a dataframe, specifying the index and the values we want to represent on the choropleth.

We then link our dataframe with the state co-ordinates in the geojson file, producing a visual representation of 2017 gun violence deaths per 100,000 people by state:

In [13]:
n_killed_map.choropleth(geo_data='us-states_processed.json', 
                        data=choro_df, 
                        columns =['state', 'killed per 100000'], 
                        key_on='feature.id', fill_color='YlOrRd', 
                        legend_name='No. killed per 100,000 in 2017'
                        )
n_killed_map

The 2nd choropleth shows the number of gun violence incidents (deaths or injuries) per 100,000 people:

In [14]:
n_incidents_map = folium.Map(location=[38.0215482, -95.471191], tiles='OpenStreetMap', zoom_start=4.3)
n_incidents_map.choropleth(geo_data='us-states_processed.json', 
                  data=choro_df, columns =['state', 'incidents per 100000'], 
                  key_on='feature.id', fill_color='PuRd', 
                  legend_name='No. incidents per 100,000 in 2017'
                  )
n_incidents_map

We can also use folium to generate a heatmap of gun deaths. To do that we simply create another map object, generate the heatmap, then add it to our map:

In [18]:
from folium.plugins import HeatMap, FastMarkerCluster

#Create new map object
hm_map = folium.Map(location=[38.0215482, -95.471191], tiles='OpenStreetMap', zoom_start=4.3)

#Prepare our data so we can pass it to the HeatMap function
hm_data = df[['latitude', 'longitude', 'n_killed']].as_matrix().tolist()

#Generate heaatmap, and overlay our new map object with the heatmap
HeatMap(hm_data).add_to(hm_map)

#Display the map
hm_map

Finally, we can visualise where any injury or death occured using the FastMarkerCluster. 

In [16]:
m_cluster = folium.Map(location=[38.0215482, -95.471191], tiles='OpenStreetMap', zoom_start=4.3)
m_cluster.add_child(FastMarkerCluster(df[['latitude', 'longitude']].as_matrix().tolist()))